# Glycine 
- [basic test](#Basic-test) 
    - [diffraction pattern](#Diffraction-pattern-for-Glycine-frame-19)
    - [padding study](#Padding-study)
- [Comparison with experiment](#Comparing-with-experimental-data)
    - [diffraction patterns vs exp](#Glycine-exp-vs-simulated-patterns)
- [Pets interface](#Pets-interface)

In [ ]:
import importlib as imp
import multislice.multislice as mupy;imp.reload(mupy)       # the temsim wrapper module
import multislice.postprocess as pp ;imp.reload(pp)         # postprocess module
import multislice.mupy_utils as mut ;imp.reload(mut)        # multislice utilities
from utils import*                         # utilities
from ipywidgets import interact, interactive, fixed, interact_manual
# %matplotlib notebook
path = 'data/glycine/'
cif_file = path+'alpha_glycine.cif'

## Basic test
### Example frame 19
The test is performed from the orientation of the experimental diffraction pattern 19. 

In [ ]:
xyz = 'alpha_glycine_test.xyz'
n = array([0.86956126,-11.24280557,-0.34604196])
mut.gen_xyz2(cif_file,n=n,theta=0,lat_params=[100,200,100],pad=0.5,xyz=path+xyz)
mut.show_grid(path+xyz,opts=['xy','xz'],popts='h')

In [ ]:
crys  = mut.import_crys(file)
crys.lattice_parameters

In [ ]:
args = {'name':path,'mulslice':False,'data':xyz,'keV':200,'i_slice':10,
             'repeat':[1,1,1],'NxNy':4096,'slice_thick':1.0,'Nhk':1}
multi = mupy.Multislice(tag='test', opt='srfp',ppopt='s',ssh='badb',**args)

In [ ]:
multi = pp.load(path,tag='0',v=1)#;multi.print_log()

In [ ]:
pargs = {'Iopt':'csng','Imax':5e4,'gs':0.015,'rmax':25,'Nmax':512,'xylims':1.5,'cmap':'viridis'}
multi.pattern(iz=4,v=1,caxis=[0,1e-3],name='figures/glycine19.png',opt='ps',**pargs);
# multi.patterns2gif(**pargs,caxis=[0,1e-5])

In [ ]:
# %matplotlib notebook
iz=20
# multi.save_pattern(iz)
pargs = {'Iopt':'csng','gs':0.015,'rmax':25,'Nmax':512,'rings':[1],'xylims':1.2,'cmap':'viridis'}
qx,qy,im0 = multi.pattern(iz=iz,out=1,lw=3,**pargs);
t = np.linspace(0,2*np.pi,100)
ct,st = np.cos(t),np.sin(t)
plts = [[r*ct,r*st,'g--','',1] for r in np.arange(0.25,1.5,0.25)]
plts+= [[r*ct,r*st,'g-','',2] for r in range(1,2)]
fig,ax = dsp.create_fig()
def faxis(x):
    print('vmax=',float(x))
    ax.cla();dsp.stddisp(plts,ax=ax,fig=fig,im=[qx,qy,im0],title='glycine',pOpt='im',caxis=[0,float(x)])
    print('ok')
interact(faxis, x="0.001");

### Diffraction pattern for Glycine frame 19 

exp | multislice
--- | ---------
[![](figures/pattern_19.png)](figures/pattern_19.png) | ![](figures/glycine19.png)

<!-- Thickness dependent diffraction pattern gif   -->

<!-- gif | pattern -->
<!-- --- | ------ -->
<!-- ![](data/glycine/figures/glycine_0_autoslic_pattern.gif) | ![](figures/glycine19.png) -->

### Padding study


In [ ]:
pads,lats = np.meshgrid([0.5,1,2],[50,100,200])
pads,lats = pads.flatten(),lats.flatten()
for pad,lat in zip(pads,lats):
    

## Comparing with experimental data
### Example on multiple frames
A $20\times20$ crystal is considered although due to orienting the crystal it is not possible to garantee the exact number. 

In [ ]:
frames = np.arange(1,65) #[5,10,15,25,30]
df = pd.DataFrame(frames,columns=['frame'])
uvw = np.load(path+'uvw.npy')

df['uvw']=[u for u in uvw]
for f,n in zip(frames,uvw):
    xyz = 'alpha_glycine_test%d.xyz' %f
    mut.gen_xyz2(cif_file,n=n,theta=0,lat_params=[100,100,100],pad=0.5,xyz=path+xyz)
#     mut.show_cell(cif_file,n=n,bopt=1,x0=None,rep=[1,1,1])
    # mut.show_grid(path+xyz,opts=['xy','xz'],popts='h')

In [ ]:
args = {'name':path,'mulslice':False,'keV':200,'i_slice':10,
             'repeat':[1,1,1],'NxNy':4096,'slice_thick':1.0,'Nhk':1}
frames=np.arange(1,10)#[5,10,15,25,30])
for f in zip(frames):
    xyz = 'alpha_glycine_test%d.xyz' %f
    mupy.Multislice(tag='test%d' %f,data=xyz, opt='srfp',ppopt='s',ssh='badb',**args);

In [ ]:
frames=np.arange(1,65)#[5,10,15,25,30])
m = np.zeros(frames.shape,dtype=object)
pargs = {'Iopt':'csngt','Imax':5e4,'gs':0.015,'rmax':25,'Nmax':512,'xylims':1.5,'cmap':'viridis'}
for i,f in enumerate(frames):
    m[i] = pp.load(path,'test%d' %f,v=1)
#     m[i].pattern(opt='c',caxis=[0,1e-3],name='figures/glycine_test%d.png' %f,**pargs)

### Glycine exp vs simulated patterns 

05 | 10 | 15 | 25 | 30
-- | -- | -- | -- | --
![](figures/glycine_test5.png) | ![](figures/glycine_test10.png) | ![](figures/glycine_test15.png) | ![](figures/glycine_test25.png) | ![](figures/glycine_test30.png)
![](figures/glycine_exp5.png) | ![](figures/glycine_exp10.png) | ![](figures/glycine_exp15.png) | ![](figures/glycine_exp25.png) | ![](figures/glycine_exp30.png)


## Pets interface
From the outputs produced by pets, the `pets` module can be used as a python interface.

The full path of the PETS2 configuration file (**.pts** file) must be specified.The containing folder should also contain the output from pets.

The experimental frames if available can immediately be visualized with `show_exp` (the interactivity of the figure does not work well with jupyter).

In [ ]:
from multislice import pets as pets
%matplotlib notebook
pets = pets.Pets('data/glycine/pets/glycine.pts')
pets.show_exp(frame=19);

### Retrieving the experimental orientation 

#### Theory

Following [dials convention](https://dials.github.io/documentation/conventions.html), the following relationship holds in cartesian reciprocal space : 

$$\mathbf r_{\phi} = \mathbf R\mathbf A\mathbf h$$

where $\mathbf R$ is the rotation matrix, $\mathbf A=\mathbf U\mathbf B$ the crystal orientation matrix for a rotation of $0^{\circ}$ and 
$\mathbf h=h\mathbf a^{*}+k\mathbf b^{*}+l\mathbf c^{*}$.


#### Diffraction patterns to reciprocal space

The reflections from diffraction patterns are converted from each 2D plane into cartesian reciprocal space from : 

$$\mathbf r_{\phi}=\mathbf D \left(\mathbf p_{xy} - \mathbf c_{xy}\right)a_{px}$$

where $\mathbf D$ is the lab coordinate matrix, $\mathbf p_{xy}$ are the pixel positions of the reflections,  $\mathbf c_{xy}$ is the pixel position of the central beam, and $a_{px}=p_x/\lambda D$ the reciprocal aperture per pixel.

#### Indexing 

The miller indices are obtained from the indexing matrix $\mathbf A^{-1}$ :

$$\mathbf h = \mathbf A^{-1}\mathbf r_{lp}$$

#### Using the output from pets

In [ ]:
pets.show_xyz()

crystal rotation Orientation [x](figures/x.mp4), [y](figures/y.mp4) , [z](figures/z.mp4).

<!-- jupyter | pets -->
<!-- ------- | ----- -->
<!-- ![](figures/glycine_orient.png) | ![](figures/glycine_pets_orient.png) -->
